□ Logistic 회귀를 이용한 이진 분류
1. dcsn과 proba의 관계
proba의 첫 번째 열은 음성 클래스(0)에 대한 확률이고, 두 번째 열은 양성 클래스(1)에 대한 확률이다.
dcsn은 z값을 갖는데 이를 시그모이드 함수에 통과시키면 양성 클래스일 확률이 나오게 된다.
따라서 'dcsn == proba의 두 번째열'이 성립한다.

2. predict_proba 사용하지 않고 proba 얻기

In [15]:
import pandas as pd
import numpy as np
# 사용할 데이터 불러오기
fish = pd.read_csv('https://bit.ly/fish_csv')
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()
# , 학습데이터 테스트데이터 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
 fish_input, fish_target, random_state=42)
# 특성 정규화
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)
# : Bream Smelt ! 이진분류를 위해 데이터 골라내기 과 만 분류해 보자
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
#
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)
# decision_function 값
dcsn = lr.decision_function(train_bream_smelt[:5])
print(dcsn)
# 각 클래스에 속할 확률
proba = lr.predict_proba(train_bream_smelt[:5])
print(proba)
print()

#predict_proba 사용하지 않고 proba 얻기
from scipy.special import expit
proba_1 = expit(dcsn)
proba_0 = [1-p1 for p1 in proba_1]

mk_proba = np.column_stack([proba_0, proba_1])
print(mk_proba)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]
[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


□ Logistic 회귀를 이용한 다중 분류
3.  위 코드를 실행시켜 와 값이 일치함을 확인하시오

In [16]:
# 다중분류를 위한 모델과 학습
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
# 예측
print(lr.predict(test_scaled[:3]))
# 3 (predict_proba) 테스트 데이터 개가 각 클래스에 속할 확률
proba = lr.predict_proba(test_scaled[:3])
print(np.round(proba, decimals=3))
# decision_function 값들
decision = lr.decision_function(test_scaled[:3])
print(np.round(decision, decimals=2))
# softmax predict_proba 함수를 이용한 계산
from scipy.special import softmax
proba_sm = softmax(decision, axis=1)
print(np.round(proba_sm, decimals=3))

['Perch' 'Smelt' 'Pike']
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]
[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]]
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]


4.  다음의 교제 정의를 이용하여 을 계산하는 코드를 작성하시오

In [35]:
# 다중분류를 위한 모델과 학습
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
# 예측
#print(lr.predict(test_scaled[:3]))
# 3 (predict_proba) 테스트 데이터 개가 각 클래스에 속할 확률
proba = lr.predict_proba(test_scaled[:3])
#print(np.round(proba, decimals=3))
# decision_function 값들
decision = lr.decision_function(test_scaled[:3])
print(np.round(decision, decimals=2))
# softmax predict_proba 함수를 이용한 계산
from scipy.special import softmax
proba_sm = softmax(decision, axis=1)
print(np.round(proba_sm, decimals=3))

#===============
#직접 proba_sm 구하기
e_all = [np.exp(s) for s in decision] #모두 e^zi로 전환

e_sum = [sum(e_all[i]) for i in range(3)] #esum 구하기

mk_proba_sm = [] #si값들
for i in range(3):
  for j in range(7):
    mk_proba_sm.append(e_all[i][j]/e_sum[i])

print("직접 계산한 proba_sm")
new_proba_sm = np.round(mk_proba_sm, decimals=3).reshape(3,7)
print(new_proba_sm)

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]]
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]
직접 계산한 proba_sm
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]
